In [2]:
'''If gives out error, ignore'''
from pyspark import SparkContext
sc=SparkContext()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=PySparkShell, master=local[*]) created by <module> at /usr/local/lib/python2.7/dist-packages/IPython/utils/py3compat.py:217 

In [6]:
TWEET_PATH = '/home/victor/Desktop/Bigdatathon/bigdatathon-2017/data/'
TWEET_FILE = 'tweetssentiment.txt'
TWEET_PATHFILE = TWEET_FILE
ITEM_PATH = '/home/victor/Desktop/Bigdatathon/bigdatathon-2017/scrapers/farfetch/'
ITEM_FILE = 'kv-data-hashed-forvictor'
ITEM_PATHFILE = ITEM_FILE



Tweets = (sc.textFile(TWEET_PATHFILE, 4).distinct().map(lambda x : x.split(',')).cache())


In [7]:
Items = (sc.textFile(ITEM_PATHFILE, 4).map(lambda x : x.split('","')).cache())

In [8]:
stopwords = set(["a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along",
                  "already", "also","although","always","am","among", "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone",
                  "anything","anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become","becomes", "becoming",
                  "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom",
                  "but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done",
                  "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every",
                  "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly",
                  "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her",
                  "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if",
                  "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many",
                  "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself",
                  "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing",
                  "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our",
                  "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem",
                  "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some",
                  "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the",
                  "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv",
                  "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward",
                  "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were",
                  "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while",
                  "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you",
                  "your", "yours", "yourself", "yourselves", "the","add"]);


In [9]:
import re
split_regex = r'[a-zA-Z0-9_]+'
def simpleTokenize(string):
    """ A simple implementation of input string tokenization
    Args:
        string (str): input string
    Returns:
        list: a list of tokens
    """
    string = re.sub(r'([^\s\w(?<_)])+', ' ',string.lower())
    return re.findall(split_regex, string)


def removeQuotes(s):
    """ Remove quotation marks from an input string
    Args:
        s (str): input string that might have the quote "" characters
    Returns:
        str: a string without the quote characters
    """
    return ''.join(i for i in s if i!='"')

def tokenize(string):
    """ An implementation of input string tokenization that excludes stopwords
    Args:
        string (str): input string
    Returns:
        list: a list of tokens without stopwords
    """
    string = removeQuotes(string)
    def remove_if_not_substring(l1, l2):
        return [i for i in l1 if not any(j == i for j in list(l2))]
    string = remove_if_not_substring(simpleTokenize(string), stopwords)
    return string



TweetsPARSED = Tweets.map(lambda x: (x[1],(x[0],tokenize(x[2]),x[3])))
ItemsPARSED = Items.map(lambda x: (removeQuotes(x[9]),(removeQuotes(x[2]),re.sub('\s+','_', x[3]),removeQuotes(x[4]),removeQuotes(tokenize(x[5])))))
##TweetsPARSED = Tweets.map(lambda x: (removeQuotes(x[1]),1))
##ItemsPARSED = Items.map(lambda x: (removeQuotes(x[9]),1))
TweetsPARSED.take(2)

[(u'1ef7556e0472eef35c1b0e3356674551',
  (u'saviano_lucia',
   [u'dolce', u'notte', u'sonia', u'grazie', u'te'],
   u'0.0')),
 (u'65ed672af933c88c445fed9e78fa461c',
  (u'Theawaster95',
   [u'love',
    u'android',
    u'tom',
    u'ford',
    u'models',
    u'round',
    u'frame',
    u'fashion',
    u'glasses',
    u'black',
    u'img',
    u'stylewidth',
    u'myfirsttweet'],
   u'0.6369'))]

In [10]:
TweetsPARSED.join(ItemsPARSED).mapValues(lambda x: x[0] + (x[1], )).take(2)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 3.0 failed 1 times, most recent failure: Lost task 7.0 in stage 3.0 (TID 12, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/worker.py", line 101, in main
    process()
  File "/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/worker.py", line 96, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/serializers.py", line 236, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-9-ab729c1ee7a5>", line 39, in <lambda>
IndexError: list index out of range

	at org.apache.spark.api.python.PythonRDD$$anon$1.read(PythonRDD.scala:135)
	at org.apache.spark.api.python.PythonRDD$$anon$1.ne

In [11]:
from pyspark.sql.types import Row

col_names_tweets = ['ItemID', 'UID', 'Tweet', 'sent_score']
col_names_items = ['ItemID', 'ItemNAME', 'ItemDESIGNER', 'ItemPRICE', 'ItemDESC']
def f(x,col_names):
    d = {}
    for i in range(len(x)):
        d[col_names[i]] = x[i]
    return d

TweetsDF = TweetsPARSED.map(lambda x: Row(**f(x,col_names_tweets))).toDF()
ItemsDF = ItemsPARSED.map(lambda x: Row(**f(x,col_names_items))).toDF()

In [12]:
import pandas as pd
print(TweetsDF)
print(TweetsDF.join(ItemsDF))
print(TweetsDF.toPandas())

ImportError: No module named pandas

In [88]:
TweetsDF.join(ItemsDF, TweetsDF.ItemID==ItemsDF.ItemID, 'outer').show(n=10)

Py4JJavaError: An error occurred while calling o1573.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 131.0 failed 1 times, most recent failure: Lost task 3.0 in stage 131.0 (TID 165, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/worker.py", line 101, in main
    process()
  File "/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/worker.py", line 96, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/serializers.py", line 236, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-72-ab729c1ee7a5>", line 39, in <lambda>
IndexError: list index out of range

	at org.apache.spark.api.python.PythonRDD$$anon$1.read(PythonRDD.scala:135)
	at org.apache.spark.api.python.PythonRDD$$anon$1.next(PythonRDD.scala:98)
	

In [65]:
Count = (ItemsPARSED.join(TweetsPARSED)).cache()
Count.count()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 85.0 failed 1 times, most recent failure: Lost task 3.0 in stage 85.0 (TID 122, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/worker.py", line 101, in main
    process()
  File "/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/worker.py", line 96, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/serializers.py", line 236, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-60-77c3f307847d>", line 41, in <lambda>
IndexError: list index out of range

	at org.apache.spark.api.python.PythonRDD$$anon$1.read(PythonRDD.scala:135)
	at org.apache.spark.api.python.PythonR

In [62]:
print(ItemsPARSED.take(10))
print(TweetsPARSED.take(1))

[(u'e15ac260bca0119df0eccc5817d755dd', 1), (u'93a2ee10badc5938b6f5e9dee9d42824', 1), (u'5977684c389327cc46ef05b0b9b0b516', 1), (u'ad8a2bd0894e12c5429fefd9912e6261', 1), (u'2de71118b566d50d98da59a4bbbed57a', 1), (u'6f58b9d9827cf97b693fc1c00b7aece9', 1), (u'7cba40a3927a513c984aa99d80b56069', 1), (u'1f1800f46b6470001bd7b2eae3e6777d', 1), (u'22d80aeb4aeb2ef395a4d55f22ef9631', 1), (u'5b8cc7f0167a0a612a1dbbf25ff5aa35', 1)]
[(u'1ef7556e0472eef35c1b0e3356674551', 1)]


In [63]:
a = ItemsPARSED.join(TweetsPARSED)
b = ItemsPARSED + TweetsPARSED

In [64]:
a.take(2)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 82.0 failed 1 times, most recent failure: Lost task 3.0 in stage 82.0 (TID 117, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/worker.py", line 101, in main
    process()
  File "/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/worker.py", line 96, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/serializers.py", line 236, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-60-77c3f307847d>", line 41, in <lambda>
IndexError: list index out of range

	at org.apache.spark.api.python.PythonRDD$$anon$1.read(PythonRDD.scala:135)
	at org.apache.spark.api.python.PythonRDD$$anon$